## Reflection

Go provides a mechanism to update variables and inspect their values at run time, to call their methods, and to apply the operations intrinsic to their representation, all without knowing their types at compile time. This mechanism is called reflection.

In [2]:
import "strconv"

func Sprint(x interface{}) string {
    type stringer interface {
        String() string
    }
    switch x := x.(type) {
    case stringer:
        return x.String()
    case string:
        return x
    case int:
        return strconv.Itoa(x)
    // ...similar cases for int16, uint32, and so on...
    case bool:
        if x {
            return "true"
        }
        return "false"
    default:
        // array, chan, func, map, pointer, slice, struct
        return "???"
    }
}


In [3]:
Sprint(1)

1

In [4]:
Sprint(true)

true

In [5]:
Sprint("this is a string")

this is a string

## reflect.Type and reflect.Value

In [10]:
import (
    "fmt"
    "reflect"
)

t := reflect.TypeOf(3)  // a reflect.Type
t

int

In [11]:
fmt.Println(t.String())

int


4 <nil>

In [12]:
reflect.TypeOf(t)

*reflect.rtype

Recall from Section 7.5 that an assignment from a concrete value to an interface type performs an implicit interface conversion, which creates an interface value consisting of two components: its dynamic type is the operand’s type (int) and its dynamic value is the operand’s value (3).

In [15]:
import (
    "io"
    "os"
)

var w io.Writer = os.Stdout
fmt.Println(reflect.TypeOf(w))

*os.File


9 <nil>

In [17]:
v := reflect.ValueOf(3)
v

3

In [18]:
reflect.TypeOf(v)

reflect.Value

A reflect.Value and an interface{} can both hold arbitrary values. The difference is that an empty interface hides the representation and intrinsic operations of the value it holds and exposes none of its methods, so unless we know its dynamic type and use a type assertion to peer inside it (as we did above), there is little we can do to the value within. In contrast, a Value has many methods for inspecting its contents, regardless of its type. 

## Setting Variables with reflect.Value

In [24]:
x := 2

In [33]:
a := reflect.ValueOf(2)
a

2

In [34]:
b := reflect.ValueOf(x)
b

2

In [28]:
c := reflect.ValueOf(&x)
c

0xc000494000

In [29]:
d := c.Elem()
d

2

We can ask a reflect.Value whether it is addressable through its CanAddr method.

In [35]:
a.CanAddr()

false

In [36]:
b.CanAddr()

false

In [37]:
c.CanAddr()

false

In [38]:
d.CanAddr()

true

To recover the variable from an addressable reflect.Value requires three steps. First, we call Addr(), which returns a Value holding a pointer to the variable. Next, we call Interface() on this Value, which returns an interface{} value containing the pointer. Finally, if we know the type of the variable, we can use a type assertion to retrieve the contents.

In [39]:
x := 2

In [42]:
d := reflect.ValueOf(&x).Elem()  // d refers to the variable x
d

2

In [44]:
d.Addr()

0xc000494000

In [43]:
px := d.Addr().Interface().(*int)   // px := &x
px

0xc000494000

In [45]:
*px = 3

In [46]:
x

3

In [47]:
d.Set(reflect.ValueOf(4))
x

4

There are variants of Set specialized for certain groups of basic types: SetInt, SetUint, SetString, SetFloat, and so on.

## Displaying the Methods of a Type

In [49]:
import "strings"

// Print prints the method set of the value x.
func Print(x interface{}) {
    v := reflect.ValueOf(x)
    t := v.Type()
    fmt.Printf("type %s\n", t)

    for i := 0; i < v.NumMethod(); i++ {
        methType := v.Method(i).Type()
        fmt.Printf("func (%s) %s%s\n", t, t.Method(i).Name,
            strings.TrimPrefix(methType.String(), "func"))
    }
}

In [50]:
type Dog struct {
    name string
}

In [51]:
func (d *Dog) GetName() string {
    return d.name
}

func (d *Dog) Bark() {
    fmt.Println("Woof!")
}

In [53]:
d := Dog{"Fido"}

In [54]:
d.GetName()

Fido

In [55]:
d.Bark()

Woof!


In [56]:
Print(d)

type struct { 𒀸name string }


## A word of caution

There is a lot more to the reflection API than we have space to show, but the preceding examples give an idea of what is possible. Reflection is a powerful and expressive tool, but it should be used with care.

***